## A convolutional autoencoder

Interesting note:
This model performs better on representing the data than the MLP AE. 
It is less sensitive to overfitting, because of the Dropout layers. 

When comparing the mean and standard deviation of input data and reproduced data, 
this model comes much closer to the original than the MLP does.


The model looks like this:

```_______________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         (None, 32, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 32, 18)            54        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 18)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 16, 8)             296       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 8, 8)              0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 8, 8)              136       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 4, 8)              0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 4, 8)              136       
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 8, 8)              0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 8, 8)              136       
_________________________________________________________________
up_sampling1d_2 (UpSampling1 (None, 16, 8)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 16, 18)            306       
_________________________________________________________________
up_sampling1d_3 (UpSampling1 (None, 32, 18)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 32, 1)             37        
=================================================================
Total params: 1,101
Trainable params: 1,101
Non-trainable params: 0```

In [13]:
# imports

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D
from keras.models import Model
import keras.backend as K
from keras.callbacks import EarlyStopping
import time

import numpy as np
from music21 import midi, stream

# Importing functions used in AE Midi
from create_m21_object import createMusic21Object
from create_pattern import createPattern

from music21 import converter, instrument, note, chord
from music21.instrument import Guitar
from music21 import midi, stream

from sklearn.preprocessing import MinMaxScaler

# For training
epochs = 400
batch_size = 64

In [2]:
# Get data
data = np.load("music.npz")
x_train = data['x_train']
x_test = data['x_test']

# Reshaping to use in a Conv1D network
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

x_train.shape, x_test.shape

((11292, 32, 1), (5645, 32, 1))

In [3]:
input_song = Input(shape=(32,1)) # channels last

### Encoder model

In the encoder model, each layer halfens the size of the input, ending up with a bottleneck of size 4.

In [4]:
# create encoder model

x = Conv1D(filters=18, kernel_size=(2), activation='relu', padding='same')(input_song)
x = MaxPooling1D((2), padding='same')(x)
x = Conv1D(filters=8, kernel_size=(2), activation='relu', padding='same')(x)
x = MaxPooling1D((2), padding='same')(x)
x = Conv1D(8, (2), activation='relu', padding='same')(x)
encoded = MaxPooling1D((2), padding='same')(x)

### Decoder model

In the decoder model, each layer doubles the size of its input, ending up with the output size equal to the input size.

In [5]:
# create decoder model

x = Conv1D(8, (2), activation='relu', padding='same')(encoded)
x = UpSampling1D((2))(x)
x = Conv1D(8, (2), activation='relu', padding='same')(x)
x = UpSampling1D((2))(x)
x = Conv1D(18, kernel_size=(2), activation='relu', padding='same')(x)
x = UpSampling1D((2))(x)
decoded = Conv1D(1, (2), activation='sigmoid', padding='same')(x)

In [7]:
autoencoder = Model(input_song, decoded)
autoencoder.compile(optimizer='rmsprop', loss='mean_squared_error')

earlystop = EarlyStopping(monitor='val_loss', min_delta=10e-5, patience=5,
                          verbose=1, mode='min')

callbacks_list = [earlystop]

# train the model
start = time.time()

autoencoder.fit(x_train, x_train,
               epochs=epochs,
               batch_size=batch_size,
               shuffle=True,
               validation_split=0.2)

end = time.time()

print("time to train", end-start)

Train on 9033 samples, validate on 2259 samples
Epoch 1/400
9033/9033 [==============================] - 1s 159us/step - loss: 0.0350 - val_loss: 0.0397
Epoch 2/400
9033/9033 [==============================] - 1s 113us/step - loss: 0.0349 - val_loss: 0.0412
Epoch 3/400
9033/9033 [==============================] - 1s 118us/step - loss: 0.0349 - val_loss: 0.0407
Epoch 4/400
9033/9033 [==============================] - 1s 112us/step - loss: 0.0349 - val_loss: 0.0410
Epoch 5/400
9033/9033 [==============================] - 1s 115us/step - loss: 0.0348 - val_loss: 0.0416
Epoch 6/400
9033/9033 [==============================] - 1s 114us/step - loss: 0.0348 - val_loss: 0.0420
Epoch 7/400
9033/9033 [==============================] - 1s 115us/step - loss: 0.0348 - val_loss: 0.0399
Epoch 8/400
9033/9033 [==============================] - 1s 117us/step - loss: 0.0348 - val_loss: 0.0390
Epoch 9/400
9033/9033 [==============================] - 1s 118us/step - loss: 0.0348 - val_loss: 0.0400
Epoch 1

9033/9033 [==============================] - 1s 100us/step - loss: 0.0337 - val_loss: 0.0385
Epoch 79/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0337 - val_loss: 0.0388
Epoch 80/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0336 - val_loss: 0.0380
Epoch 81/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0337 - val_loss: 0.0387
Epoch 82/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0336 - val_loss: 0.0401
Epoch 83/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0336 - val_loss: 0.0382
Epoch 84/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0336 - val_loss: 0.0390
Epoch 85/400
9033/9033 [==============================] - 1s 104us/step - loss: 0.0336 - val_loss: 0.0384
Epoch 86/400
9033/9033 [==============================] - 1s 99us/step - loss: 0.0336 - val_loss: 0.0382
Epoch 87/400
9033/9033 [==============================] - 1s

9033/9033 [==============================] - 1s 102us/step - loss: 0.0327 - val_loss: 0.0376
Epoch 156/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0327 - val_loss: 0.0383
Epoch 157/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0327 - val_loss: 0.0372
Epoch 158/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0326 - val_loss: 0.0389
Epoch 159/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0326 - val_loss: 0.0376
Epoch 160/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0326 - val_loss: 0.0373
Epoch 161/400
9033/9033 [==============================] - 1s 99us/step - loss: 0.0326 - val_loss: 0.0378
Epoch 162/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0326 - val_loss: 0.0388
Epoch 163/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0325 - val_loss: 0.0371
Epoch 164/400
9033/9033 [===========================

Epoch 232/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0314 - val_loss: 0.0365
Epoch 233/400
9033/9033 [==============================] - 1s 105us/step - loss: 0.0314 - val_loss: 0.0377
Epoch 234/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0314 - val_loss: 0.0384
Epoch 235/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0314 - val_loss: 0.0357
Epoch 236/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0314 - val_loss: 0.0370
Epoch 237/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0313 - val_loss: 0.0365
Epoch 238/400
9033/9033 [==============================] - 1s 104us/step - loss: 0.0313 - val_loss: 0.0378
Epoch 239/400
9033/9033 [==============================] - 1s 106us/step - loss: 0.0313 - val_loss: 0.0370
Epoch 240/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0313 - val_loss: 0.0365
Epoch 241/400
9033/9033 [============

9033/9033 [==============================] - 1s 102us/step - loss: 0.0305 - val_loss: 0.0351
Epoch 309/400
9033/9033 [==============================] - 1s 99us/step - loss: 0.0305 - val_loss: 0.0355
Epoch 310/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0304 - val_loss: 0.0363
Epoch 311/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0305 - val_loss: 0.0350
Epoch 312/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0304 - val_loss: 0.0357
Epoch 313/400
9033/9033 [==============================] - 1s 105us/step - loss: 0.0304 - val_loss: 0.0362
Epoch 314/400
9033/9033 [==============================] - 1s 99us/step - loss: 0.0305 - val_loss: 0.0350
Epoch 315/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0304 - val_loss: 0.0353
Epoch 316/400
9033/9033 [==============================] - 1s 105us/step - loss: 0.0304 - val_loss: 0.0372
Epoch 317/400
9033/9033 [============================

Epoch 385/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0300 - val_loss: 0.0350
Epoch 386/400
9033/9033 [==============================] - 1s 100us/step - loss: 0.0299 - val_loss: 0.0346
Epoch 387/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0299 - val_loss: 0.0359
Epoch 388/400
9033/9033 [==============================] - 1s 102us/step - loss: 0.0299 - val_loss: 0.0346
Epoch 389/400
9033/9033 [==============================] - 1s 101us/step - loss: 0.0299 - val_loss: 0.0384
Epoch 390/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0299 - val_loss: 0.0353
Epoch 391/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0299 - val_loss: 0.0347
Epoch 392/400
9033/9033 [==============================] - 1s 103us/step - loss: 0.0299 - val_loss: 0.0349
Epoch 393/400
9033/9033 [==============================] - 1s 105us/step - loss: 0.0299 - val_loss: 0.0359
Epoch 394/400
9033/9033 [============

In [11]:
# Importing feature range from MLP example
feature_range = np.load("feature_range.npy")

# Creating a scaler for the predicted values
predictScaler = MinMaxScaler(feature_range=feature_range)

song_prediction = autoencoder.predict(x_test)

# Reshape to allow scaler to work
song = song_prediction.reshape((song_prediction.shape[0], song_prediction.shape[1]))

song = (predictScaler.fit_transform(song)).astype('int')
newsong = createPattern(song[40])

play = createMusic21Object(newsong)

In [19]:
mt = midi.MidiTrack(0)
dt = midi.DeltaTime(mt)
dt.time = 0.5
s1 = stream.Stream()

offset = 0
for item in newsong:
    
    if ('.' in item) or item.isdigit():
        # chord
        notes_in_chord = item.split('.')
        notes = []

        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Guitar()
            new_note.offset = offset
            notes.append(new_note)

        new_chord = chord.Chord(notes)
        s1.append(new_chord)

    elif item is not '' and ('.' not in item):
        # notes
        new_note = note.Note(item)
        s1.append(new_note)


    elif item == '':
        # rest
        s1.append(note.Rest())

In [18]:
midi_stream = stream.Stream(s1)

sp = midi.realtime.StreamPlayer(midi_stream)

sp.play()

In [16]:
song_prediction.mean(), x_test.mean(), song_prediction.std(), x_test.std()
# autoencoder.summary()

(0.8418441, 0.8293800943207219, 0.19854255, 0.26314332054883544)